In [6]:
from pathlib import Path
import shutil # shutil is still useful for high-level file operations like copy2

src_root = Path('data/sim_cases')
dst_root = Path('data/finit_files')

# Create destination directory if it doesn't exist
dst_root.mkdir(parents=True, exist_ok=True) # pathlib's mkdir is very convenient

finit_file_names = []

# Iterate through each simulation case folder
# Using .iterdir() directly on the Path object
for case_path in src_root.iterdir():
    # .is_dir() is a method on Path objects
    if case_path.is_dir():
        # Find the finit file (case-insensitive, endswith .FINIT)
        # Iterate over files within the case_path directory
        for src_file in case_path.iterdir():
            # .is_file() to ensure it's a file, not a subdirectory
            # .suffix gives the extension, .upper() for case-insensitivity
            if src_file.is_file() and src_file.suffix.upper() == '.FINIT':
                # .name gives just the filename (e.g., 'CASE_A.FINIT')
                dst_file = dst_root / src_file.name # Use / operator to join Path objects
                shutil.copy2(src_file, dst_file)
                finit_file_names.append(src_file.name) # Append the file name

# Print the list of finit file names
# print('Copied FINIT files:')
# for fname in finit_file_names:
#     print(fname)
print(f'Total FINIT files copied: {len(finit_file_names)}')

Total FINIT files copied: 0


In [2]:
finit_file_names

['JD_BASECASE_2.FINIT', 'JD_BASECASE_1.FINIT']

In [1]:
finit_file_names=['JD_BASECASE_26.FINIT']

In [ ]:
from pathlib import Path
from extract_properties_from_finit import extract_properties_from_finit
from generate_full_properties import generate_full_properties
from CMG_format_compress import CMG_format_compress

finit_dir = 'data/finit_files2'
save_dir = 'results/properties'

finit_dir = Path(finit_dir)
save_dir = Path(save_dir)
save_dir.mkdir(exist_ok=True)

for finit_file_name in finit_file_names:
    finit_file_path = finit_dir / finit_file_name
    
    # STEP 1: Extract properties from FINIT file for active cells only
    extracted_property_dict = extract_properties_from_finit(
        finit_file_path = finit_file_path,
        keywords = ['PORO', 'PERMX', 'ACTID'],
        is_save = False,
        save_dir = save_dir,
        save_name = finit_file_name.split('.')[0],
        show_summary = False
    )

    # STEP 2: Generate full properties for all cells (filled inactive cells with zeros)
    full_property_dict = generate_full_properties(
        property_dict = extracted_property_dict,
        property_list = ['PORO', 'PERMX'], 
        total_cells = 989001,
        is_save = True,
        save_dir = save_dir,
        save_name = finit_file_name.split('.')[0],
        # save_name = finit_file_name.split('.')[0]+'_jreversed',
        show_summary = False,
        is_j_reversed = False,
        grid_shape = (107, 117, 79),
        )


    # STEP 3: Compress full properties to CMG format (repeated values as N*value)
    for key in ['PORO', 'PERMX']:
        CMG_format_compress(
            array = full_property_dict[key], 
            keyword = key, 
            max_line_length = 80,
            show_summary = True,
            save_dir = save_dir,
            save_name = finit_file_name.split('.')[0]
            # save_name = finit_file_name.split('.')[0]+'_jreversed'
        )



COMPRESSED PORO SUMMARY
Compressed 989,001 values into 54393 lines
Compression ratio: 18.2:1
Total cells: 989,001
PORO - Mean: 0.082755
PORO - Min: 0.000000
PORO - Max: 0.398768
Saved compressed PORO data to: results/properties/JD_BASECASE_26_PORO.dat

COMPRESSED PERMX SUMMARY
Compressed 989,001 values into 39154 lines
Compression ratio: 25.3:1
Total cells: 989,001
PERMX - Mean: 78.739363
PERMX - Min: 0.000000
PERMX - Max: 5000.000000
Saved compressed PERMX data to: results/properties/JD_BASECASE_26_PERMX.dat


In [5]:
import numpy as np

# Define the shape
shape = (107, 117, 79)

# Load the 1D array
arr_1d = np.load('results/full_PORO.npy')

# Reshape to 3D
arr_3d = arr_1d.reshape(shape)

# Flip the j-axis (axis=1)
arr_3d_flipped = arr_3d[:, ::-1, :]

# Flatten back to 1D (row-major order, which matches the original)
arr_1d_flipped = arr_3d_flipped.flatten()

# Save the rearranged 1D array
np.save('results/full_PORO_j_reversed.npy', arr_1d_flipped)

print("Rearrangement complete. Saved as results/full_PORO_j_reversed.npy")

Rearrangement complete. Saved as results/full_PORO_j_reversed.npy


# check 1 point

In [18]:
import numpy as np
por26 = np.load('results/properties/JD_BASECASE_26_PORO.npy')
por26_jreversed = np.load('results/properties/JD_BASECASE_26_jreversed_PORO.npy')
print(por26.shape)
print(por26_jreversed.shape)

(989001,)
(989001,)


In [19]:
por26[810000]

0.19873556

In [39]:
810000/(107*117)

64.70165348670021

In [41]:
810000-107*117*64

8784

In [42]:
8784/107

82.09345794392523

In [44]:
8784-107*82

10

In [16]:
107*117*6+107*45+71

80000

In [46]:
print(107*117*64+107*82+10)
print(107*117*64+107*35+10)

810000
804971


In [19]:
arr_1d[80000]==arr_1d_flipped[82889]

True

In [20]:
print(por26[810000])
print(por26_jreversed[804971])
print(por26[810000]==por26_jreversed[804971])
print(por26_jreversed[81000])

0.19873556
0.17593764
False
0.0


0.0


In [35]:
import numpy as np
perm26 = np.load('results/properties/JD_BASECASE_26_PERMX.npy')
print(perm26.shape)
print(perm26[71000])

(989001,)
75.018707


In [34]:
import numpy as np
perm26_jreversed = np.load('results/properties/JD_BASECASE_26_PERMX_jreversed.npy')
print(perm26_jreversed.shape)
print(perm26[80000]==perm26_jreversed[82889])
print(perm26_jreversed[82889])

(989001,)
True
75.018707


# test using simple arrary

In [61]:
arr_shape=(2,3,4)
arr=np.arange(1,25)
arr

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [70]:
arr3d=arr.reshape(arr_shape[2],arr_shape[1],arr_shape[0])
arr3d

array([[[ 1,  2],
        [ 3,  4],
        [ 5,  6]],

       [[ 7,  8],
        [ 9, 10],
        [11, 12]],

       [[13, 14],
        [15, 16],
        [17, 18]],

       [[19, 20],
        [21, 22],
        [23, 24]]])

In [71]:
arr3d_reversed=arr3d[:,::-1,:]
arr3d_reversed

array([[[ 5,  6],
        [ 3,  4],
        [ 1,  2]],

       [[11, 12],
        [ 9, 10],
        [ 7,  8]],

       [[17, 18],
        [15, 16],
        [13, 14]],

       [[23, 24],
        [21, 22],
        [19, 20]]])

In [72]:
arr3d1d=arr3d.flatten()
arr3d1d

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [73]:
arr3d1d_reversed=arr3d_reversed.flatten()
arr3d1d_reversed

array([ 5,  6,  3,  4,  1,  2, 11, 12,  9, 10,  7,  8, 17, 18, 15, 16, 13,
       14, 23, 24, 21, 22, 19, 20])

# mannual j reverse

In [21]:
import numpy as np
por = np.load('results/properties/JD_BASECASE_26_PORO.npy')
por_jreversed = np.load('results/properties/JD_BASECASE_26_jreversed_PORO.npy')
print(por.shape)
print(por_jreversed.shape)

(989001,)
(989001,)


In [22]:
por_shape=(107,117,79)
por3d=por.reshape(por_shape[2],por_shape[1],por_shape[0])
por3d=por3d[:,::-1,:]
por3d_1d=por3d.flatten()

print(por[810000])
print(por3d_1d[804971])
print(por[810000]==por3d_1d[804971])
print(por3d_1d[81000])

0.19873556
0.17593764
False
0.0
